In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [14]:
with sqlite3.connect(r"C:\Users\Administrator\Desktop\DAtA SEA\bi_and_ai_group\lesson-9\data\chinook.db") as conn:
    customers = pd.read_sql("SELECT * FROM customers", con=conn)

In [35]:
with sqlite3.connect(r"C:\Users\Administrator\Desktop\DAtA SEA\bi_and_ai_group\lesson-9\data\chinook.db") as conn:
    invoice_items = pd.read_sql("SELECT * FROM invoice_items", con=conn)
invoice_items    

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


In [36]:
analysis =invoices.groupby(['CustomerId']).agg(total_amount=pd.NamedAgg(column='Total', aggfunc='sum'))
analysis = analysis.sort_values('total_amount', ascending=False)
analysis.head(5)

,total_amount
CustomerId,
6,49.62
26,47.62
57,46.62
45,45.62
46,45.62


In [29]:
full_analysis = analysis.merge(customers, how='inner', on='CustomerId')[customers['CustomerId'].isin(analysis.head(5).index)]
full_analysis['Full_name'] = full_analysis['FirstName'] + ' ' + full_analysis['LastName']
full_analysis[['CustomerId', 'Full_name', 'total_amount']]

,CustomerId,Full_name,total_amount
5,28,Julia Barnett,43.62
25,15,Jennifer Peterson,38.62
44,23,John Gordon,37.62
45,12,Roberto Almeida,37.62
56,53,Phil Hughes,37.62


In [30]:
albums

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [32]:
with sqlite3.connect(r"C:\Users\Administrator\Desktop\DAtA SEA\bi_and_ai_group\lesson-9\data\chinook.db") as conn:
    albums = pd.read_sql("SELECT * FROM albums", con=conn)
albums

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [33]:
with sqlite3.connect(r"C:\Users\Administrator\Desktop\DAtA SEA\bi_and_ai_group\lesson-9\data\chinook.db") as conn:
    tracks = pd.read_sql("SELECT * FROM tracks", con=conn)
tracks

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [39]:
# Merge invoice_items with tracks to get album information
invoice_tracks = invoice_items.merge(tracks, on='TrackId')

# Group by InvoiceId and AlbumId to check if all tracks in an album were purchased
album_purchases = invoice_tracks.groupby(['InvoiceId', 'AlbumId']).size().reset_index(name='TracksPurchased')

# Get the total number of tracks per album
total_tracks_per_album = tracks.groupby('AlbumId').size().reset_index(name='TotalTracks')

# Merge album_purchases with total_tracks_per_album to compare purchased tracks with total tracks
album_purchases = album_purchases.merge(total_tracks_per_album, on='AlbumId')

# Determine if the purchase is a full album or individual tracks
album_purchases['IsFullAlbum'] = album_purchases['TracksPurchased'] == album_purchases['TotalTracks']

# Merge with invoices to get customer information
customer_purchases = album_purchases.merge(invoices[['InvoiceId', 'CustomerId']], on='InvoiceId')

# Group by CustomerId to determine preference
customer_preferences = customer_purchases.groupby('CustomerId')['IsFullAlbum'].all().reset_index()
customer_preferences['Preference'] = customer_preferences['IsFullAlbum'].apply(lambda x: 'Full Albums' if x else 'Individual Tracks')

# Calculate percentages
preference_summary = customer_preferences['Preference'].value_counts(normalize=True) * 100
print(preference_summary)

Preference
Individual Tracks    100.0
Name: proportion, dtype: float64
